In [1]:
import undetected_chromedriver as uc
# from selenium_recaptcha_solver import RecaptchaSolver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options

import dataclasses
from typing import List
import json

# test_ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'

options = Options()

# options.add_argument("--headless")  # Remove this if you want to see the browser (Headless makes the chromedriver not have a GUI)
# options.add_argument("--window-size=1920,1080")
# options.add_argument(f'--user-agent={test_ua}')

options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")

driver = uc.Chrome(headless=False, use_subprocess=False, options=options)

# solver = RecaptchaSolver(driver=driver)

In [2]:
@dataclasses.dataclass
class Show:
    name: str
    href: str
    time: datetime
    day_of_week: str
    
# get all hrefs that contain https://lottery.broadwaydirect.com/show/
def get_all_show_urls(driver):
    driver.get('https://lottery.broadwaydirect.com/')
    shows = driver.find_elements(By.XPATH, '//a[contains(@href, "https://lottery.broadwaydirect.com/show/")]')
    show_hrefs = list(set([show.get_attribute('href') for show in shows]))
    return show_hrefs

In [3]:
def get_show_entries(driver):
    table = driver.find_element(By.TAG_NAME, "table")
    trs = table.find_elements(By.TAG_NAME, "tr")
    cur_url = driver.current_url
    cur_show = cur_url.split('/show/')[-1].strip('/')
    all_shows = []
    for tr in trs:
        show_href = None
        # get element inside tr that contains "time" in class
        tds = tr.find_elements(By.TAG_NAME, "td")
        for td in tds:
            if "time" in td.get_attribute("class"):
                show_time = td.text.strip()
                datetime_object = datetime.strptime(show_time, '%m/%d/%y %I:%M %p')
                day_of_week = datetime_object.strftime('%A')
                
            # check for href in td
            if td.find_elements(By.TAG_NAME, "a"):
                show_href = td.find_element(By.TAG_NAME, "a").get_attribute("href")
        
        if show_href is None or 'enter-lottery' not in show_href:
            continue
    
        show = Show(cur_show, show_href, datetime_object, day_of_week)
        all_shows.append(show)
    return all_shows

In [18]:
import time


def fill_entry(driver):
    # dlslot_name_first
    input_first_name = driver.find_element(By.ID, 'dlslot_name_first')
    # dlslot_name_last
    input_last_name = driver.find_element(By.ID, 'dlslot_name_last')
    # dlslot_ticket_qty
    select_ticket_qty = driver.find_element(By.ID, 'dlslot_ticket_qty')
    # dlslot_email
    input_email = driver.find_element(By.ID, 'dlslot_email')
    # dlslot_dob_month
    input_dob_month = driver.find_element(By.ID, 'dlslot_dob_month')
    # dlslot_dob_day
    input_dob_day = driver.find_element(By.ID, 'dlslot_dob_day')
    # dlslot_dob_year
    input_dob_year = driver.find_element(By.ID, 'dlslot_dob_year')
    # dlslot_zip
    input_zip = driver.find_element(By.ID, 'dlslot_zip')
    # dlslot_country
    select_country = driver.find_element(By.ID, 'dlslot_country')
    # <label for="dslot_agree">
    checkbox_agree = driver.find_element(By.XPATH, '//label[@for="dlslot_agree"]')
    # btn btn-primary
    submit_button  = driver.find_element(By.CLASS_NAME, 'btn-primary')

    input_first_name.send_keys("Jared")
    input_last_name.send_keys("Ucherek")
    select_ticket_qty.send_keys("2")
    input_email.send_keys("jared.ucherek@gmail.com")
    input_dob_month.send_keys("09")
    input_dob_day.send_keys("17")
    input_dob_year.send_keys("1996")
    input_zip.send_keys("11211")
    select_country.send_keys("USA")
    # scroll down all the way
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    checkbox_agree.click()
    submit_button.click()
    
def solve_capcha(driver):
    pass

In [14]:
show_hrefs = get_all_show_urls(driver)
available_shows = [x.split('/show/')[-1].strip('/') for x in show_hrefs]
json.dumps(available_shows)

'["aladdin", "the-lion-king", "redwood-ny", "dbh-nyc", "mj-ny", "six-ny", "wicked"]'

In [6]:
all_shows: List[Show] = []

for show_href in show_hrefs:
    driver.get(show_href)
    cur_shows = get_show_entries(driver)
    all_shows.extend(cur_shows)

In [19]:
config = json.load(open('config.json'))

In [20]:
for email, options in config.items():
    print(f"Processing {options['first_name']}")
    
    options['shows'] = set(options['shows'])
    options['show_hours'] = set(options['show_hours'])
    options['show_days'] = set(options['show_days'])
    
    for show in all_shows:
        if show.name not in options['shows']:
            print(f"Skipping {show.name}")
            continue
        if show.time.hour not in options['show_hours'] or show.day_of_week not in options['show_days']:
            print(f"Skipping {show.name} at {show.time} on {show.day_of_week}")
            continue
        
        driver.get(show.href)
        fill_entry(driver)
        # solve_capcha(driver)
        # submit
        print(f"Entered {show.name} at {show.time} on {show.day_of_week}")
        

Processing Jared
Entered six-ny at 2025-02-03 19:00:00 on Monday
Entered wicked at 2025-02-02 19:00:00 on Sunday
